In [3]:
import requests
import pandas as pd
import concurrent.futures
from dotenv import load_dotenv
import os
import time

pd.set_option('display.max_columns', None)

################################################################################
#### Load Open FEC API Key 
################################################################################

# Define the path to .env file
env_path = "/Users/sarahmathey/Desktop/DSII/final_project/Credentials/.env"

# Load the environment variables
load_dotenv(env_path)

# Access the variables
my_secret = os.getenv("open_fec_key")


In [5]:
pacs = pd.read_csv('../data/processed/pac_data_processed.csv')
pacs = pacs[(pacs['CMTE_TP'].isin(['N' ,'O' ,'Q' ,'V','W'])) & (pacs['TTL_DISB']>0)]

cmte_list = pacs.CMTE_ID.unique()

In [6]:
len(cmte_list)

7875

In [34]:
################################################################################
#### Request Data from API 
################################################################################

def fetch_cmte_financials(cmte):
    url = f"https://api.open.fec.gov/v1/committee/{cmte}/totals"
    params = {'cycle':2024
              , 'api_key': my_secret
              }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json().get('results',[])  
    else:
        print(f"Error fetching data for page {cmte}: {response.status_code}")
        return None

def main():
    results =[]
    max_workers = 5
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_cmte = {executor.submit(fetch_cmte_financials,cmte): cmte for cmte in cmte_list}
        for future in concurrent.futures.as_completed(future_to_cmte):
            response = future.result()
            if response:
                results.extend(response)
            time.sleep(0.5)

    df = pd.DataFrame(results)
    print(df.head())
    df.to_csv("data/processed/pac_financials.csv", index=False)

if __name__ == "__main__":
    main()

   all_loans_received  allocated_federal_election_levin_share  \
0                 0.0                                     0.0   
1                 0.0                                     0.0   
2                 0.0                                     0.0   
3                 0.0                                     0.0   
4                 0.0                                     0.0   

   cash_on_hand_beginning_period committee_designation  \
0                       35499.13                     U   
1                      506871.77                     B   
2                     3334129.62                     B   
3                      166058.23                     B   
4                     1013328.64                     B   

  committee_designation_full committee_id  \
0               Unauthorized    C00000489   
1    Lobbyist/Registrant PAC    C00000729   
2    Lobbyist/Registrant PAC    C00000885   
3    Lobbyist/Registrant PAC    C00000059   
4    Lobbyist/Registrant PAC    C00

In [37]:

cmte_financials_df = pd.read_csv('data/processed/pac_financials.csv')


In [38]:
cmte_financials_df.columns

Index(['all_loans_received', 'allocated_federal_election_levin_share',
       'cash_on_hand_beginning_period', 'committee_designation',
       'committee_designation_full', 'committee_id', 'committee_name',
       'committee_state', 'committee_type', 'committee_type_full',
       'contribution_refunds', 'contributions',
       'contributions_ie_and_party_expenditures_made_percent',
       'convention_exp', 'coordinated_expenditures_by_party_committee',
       'coverage_end_date', 'coverage_start_date', 'cycle', 'disbursements',
       'exp_prior_years_subject_limits', 'exp_subject_limits',
       'fed_candidate_committee_contributions',
       'fed_candidate_contribution_refunds', 'fed_disbursements',
       'fed_election_activity', 'fed_operating_expenditures', 'fed_receipts',
       'federal_funds', 'filing_frequency', 'filing_frequency_full',
       'first_f1_date', 'first_file_date', 'independent_expenditures',
       'individual_contributions', 'individual_contributions_percent',


In [40]:

cmte_financials_df.shape

(7875, 93)

In [7]:
pacs.head()

,Unnamed: 0.1,CMTE_ID,CMTE_NM,CMTE_TP,CMTE_DSGN,CMTE_FILING_FREQ,TTL_RECEIPTS,TRANS_FROM_AFF,INDV_CONTRIB,OTHER_POL_CMTE_CONTRIB,CAND_CONTRI,CAND_LOANS,TTL_LOANS_RECEIVED,TTL_DISB,TRANF_TO_AFF,INDV_REFUNDS,OTHER_POL_CMTE_REFUNDS,CAND_LOAN_REPAY,LOAN_REPAY,COH_BOP,COH_COP,DEBTS_OWED_BY,NONFED_TRANS_RECEIVED,CONTRIB_TO_OTHER_CMTE,IND_EXP,PTY_COORD_EXP,NONFED_SHARE_EXP,CVG_END_DT,cmte_id_x,n_vendors,pct_to_top_vendor,n_vendors_50_pct,n_vendors_75_pct,n_vendors_90_pct,ttl_expenditures,n_opp_vendors,pct_to_top_opp_vendor,n_opp_vendors_50_pct,n_opp_vendors_75_pct,n_opp_vendors_90_pct,ttl_opp_expenditures,CAN,IND,PAC,PTY,cmte_id_y,se_amt,Unnamed: 0,filer_committee_id_number,advertisements,bank fees,consulting,contribution,credit card payment,crypto currency,earmark,events,fundraising,other,overhead,polling,refund,salary and benefits,"sms, phonebanking, digital field",taxes,technology,travel,is_scam_pac,pct_cont_from_indiv,pct_disb_to_cand,pct_disb_to_other_pac,pct_disb_to_pty,pct_fundraising,pct_disb_to_ie
0,0,C00000059,"HALLMARK CARDS, INC. PAC (HALLPAC)",Q,B,M,95727.18,0.00,91727.18,0.0,0.0,0.0,0.0,117500.00,0.0,0.00,0.0,0.0,0.0,166058.23,144285.41,0.0,0.0,102500.00,0.0,0.0,0.0,12/31/2024,C00000059,33.0,0.008889,17.0,25.0,32.0,112500.00,1.0,0.133333,NaN,NaN,NaN,15000.00,85000.0,NaN,12500.00,NaN,NaN,NaN,0.0,C00000059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.958215,0.723404,0.106383,NaN,NaN,NaN
1,1,C00000422,AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...,Q,B,M,2096337.89,0.00,1666298.01,0.0,0.0,0.0,0.0,2621577.12,0.0,13783.30,0.0,0.0,0.0,1013328.64,488089.41,0.0,0.0,1475650.00,835000.0,0.0,0.0,12/31/2024,C00000422,232.0,0.006132,129.0,190.0,221.0,978533.35,21.0,0.000041,NaN,NaN,NaN,191633.35,493400.0,NaN,233500.00,60000.0,NaN,NaN,1.0,C00000422,NaN,28308.39,NaN,NaN,NaN,NaN,NaN,NaN,1069.14,148637.85,NaN,NaN,NaN,NaN,NaN,5518.00,NaN,NaN,0,0.794861,0.188207,0.089069,0.022887,0.001093,NaN
2,2,C00000489,D R I V E POLITICAL FUND CHAPTER 886,N,U,Q,66734.55,56873.30,0.00,0.0,0.0,0.0,0.0,26426.58,0.0,0.00,0.0,0.0,0.0,35499.13,75807.10,0.0,0.0,0.00,0.0,0.0,0.0,11/25/2024,C00000489,4.0,0.117738,3.0,3.0,3.0,6794.72,4.0,0.117738,3.0,3.0,3.0,6794.72,NaN,NaN,NaN,NaN,NaN,NaN,2.0,C00000489,200.0,NaN,NaN,250.0,NaN,NaN,NaN,NaN,NaN,800.00,5544.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,NaN,NaN,NaN,NaN,NaN
3,3,C00000729,AMERICAN DENTAL ASSOCIATION POLITICAL ACTION C...,Q,B,M,2187609.33,22478.85,2086690.47,0.0,0.0,0.0,0.0,2468347.02,0.0,14277.43,0.0,0.0,0.0,506871.77,226134.08,0.0,0.0,2398500.00,0.0,0.0,0.0,11/25/2024,C00000729,384.0,0.007217,190.0,301.0,361.0,2424877.76,13.0,0.000020,NaN,NaN,NaN,55377.76,1413500.0,NaN,956000.00,NaN,NaN,NaN,3.0,C00000729,NaN,13771.89,NaN,6400.0,NaN,NaN,NaN,NaN,32699.03,777.76,NaN,NaN,NaN,NaN,NaN,337.08,NaN,NaN,0,0.953868,0.572650,0.387304,NaN,0.013485,NaN
4,4,C00000885,INTERNATIONAL UNION OF PAINTERS AND ALLIED TRA...,Q,B,M,5525285.55,0.00,5306388.90,0.0,0.0,0.0,0.0,3244535.22,0.0,250.00,0.0,0.0,0.0,3334129.62,5614879.95,0.0,0.0,1912443.65,0.0,0.0,0.0,12/31/2024,C00000885,507.0,0.000128,270.0,370.0,472.0,2350273.65,304.0,0.000128,NaN,NaN,NaN,949630.00,793100.0,NaN,520043.65,87500.0,NaN,NaN,4.0,C00000885,NaN,300.00,12050.0,887550.0,NaN,NaN,NaN,NaN,NaN,46875.00,NaN,NaN,-16500.0,968.0,NaN,18137.00,NaN,NaN,0,0.960383,0.244442,0.160283,0.026968,NaN,NaN
